In [221]:
import findspark
findspark.init()

import datetime as dt

import pymysql
import pandas.io.sql as psql
import pandas as pd
import pyspark
from pyspark.sql.functions import to_date, month,year,dayofmonth
from pyspark.sql import SparkSession
import pyinputplus
import regex as re

In [222]:
spark = SparkSession.builder.master("local[1]").appName('Credit_card.com').getOrCreate()

In [223]:
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='password',
    database='creditcard_capstone'
)

In [224]:
cursor = connection.cursor()

In [14]:
sql = "SELECT * FROM cdw_sapp_customer "
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

C:\Users\Learner_XZHCG225\AppData\Local\Temp\ipykernel_17564\166493214.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_frame = psql.read_sql(sql, con=connection)


In [ ]:
data_frame["TIMEID"] = pd.to_datetime(data_frame["TIMEID"],format='%Y%m%d')

In [ ]:
data_frame

1) Used to check the existing account details of a customer.

In [ ]:
def customer_account():
    selected_cols = ['SSN', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'CUST_COUNTRY', 'CUST_CITY', 'CUST_EMAIL',
                 'CUST_PHONE', 'FULL_STREET_ADDRESS', 'CUST_STATE', 'CUST_ZIP', 'CREDIT_CARD_NO']
    ssn = 123456100
    result_df = data_frame.loc[data_frame['SSN'] == ssn, selected_cols]
    print(result_df)

customer_account()

 Used to modify the existing account details of a customer.

In [15]:
spark_df = spark.createDataFrame(data_frame)

C:\Spark\python\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [20]:
def module_two():
    # todo get user input
    print('Module Two:\n\tModify existing account details of a customer by entering customers credit card number\n')
    ccn = 4210653310102868
    mod2 = spark_df.select('SSN', 'FIRST_NAME', 'LAST_NAME', 'MIDDLE_NAME', 'CUST_COUNTRY', 'CUST_CITY', 'CUST_EMAIL',
                     'CUST_PHONE', 'FULL_STREET_ADDRESS', 'CUST_STATE', 'CUST_ZIP', 'CREDIT_CARD_NO').filter(
        spark_df.CREDIT_CARD_NO == ccn)
    mod2.distinct().show()
    # todo ask user what to modify
    column = input('Enter Column name value is stored in: ')
    replace = input('Enter current value from the column {}: '.format(column))  # replace with current value
    new_entry = input('Enter new value to replace old value: ')
    mod2.withColumn(column, re.replace(column, replace, new_entry)).distinct() \
        .show()

In [225]:
sql = "SELECT * FROM cdw_sapp_credit_card "
cursor.execute(sql)
result_set = cursor.fetchall()
data_frame = psql.read_sql(sql, con=connection)
pd.set_option("display.max_columns", None)

C:\Users\Learner_XZHCG225\AppData\Local\Temp\ipykernel_26772\1377666987.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_frame = psql.read_sql(sql, con=connection)


In [226]:
data_frame["TIMEID"] = pd.to_datetime(data_frame["TIMEID"],format='%Y%m%d')

In [227]:
spark_df = spark.createDataFrame(data_frame)

C:\Spark\python\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [197]:
spark_df

DataFrame[TRANSACTION_ID: bigint, CUST_CC_NO: string, CUST_SSN: bigint, BRANCH_CODE: bigint, TRANSACTION_TYPE: string, TRANSACTION_VALUE: double, TIMEID: date, month: int, year: int, days: int]

In [228]:
spark_df= spark_df.withColumn("TIMEID", to_date("TIMEID", "yyyy-MM-dd"))
spark_df= spark_df.withColumn("month", month("TIMEID"))
spark_df= spark_df.withColumn("year", year("TIMEID"))
spark_df= spark_df.withColumn("days", dayofmonth("TIMEID"))


In [ ]:
spark_df.show()

In [ ]:
spark_df.printSchema()

Used to generate a monthly bill for a credit card number for a given month and year.

In [184]:
def module_three():
    # todo get user input
    print('Module Three:\n\tGenerate a monthly bill for a credit card number for a given month and year')
    ccn = 123456698
    month = 12
    year = 2018
    mod3 = spark_df.select('CUST_SSN', 'TIMEID','TRANSACTION_TYPE',
                     'TRANSACTION_VALUE').filter(
                        spark_df.CUST_SSN == ccn).\
                        filter((spark_df.month == month)& (spark_df.year== year))
    mod3.show()
    print('Credit Card Statement for Month:{} Year:{} '.format(month, year))
    mod3.agg({'TRANSACTION_VALUE': 'sum'}).show()

In [ ]:
module_three()

Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order

In [219]:
def module_four():

    start_date = 2018-3-11
    end_date = 2018-12-20
    customer_ssn = 123456698

    mod4 = spark_df.select('TIMEID', 'TRANSACTION_TYPE', 'TRANSACTION_VALUE','CUST_SSN') \
         .filter(spark_df.CUST_SSN == customer_ssn) \
         .filter((spark_df.TIMEID==start_date)&(spark_df.TIMEID==end_date)) \
         .orderBy(year(spark_df.TIMEID).desc(), month(spark_df.TIMEID).desc(), dayofmonth(spark_df.TIMEID).desc())
    mod4.show()


In [ ]:
module_four()

In [205]:
def module_four():
    print("Transactions made by a customer between two dates.")
    ssn = 123456698
    first_date=2018-3-11
    second_date=2018-12-20
    mod4 = spark_df.select("CUST_SSN","TIMEID","TRANSACTION_TYPE","TRANSACTION_VALUE").\
        filter(spark_df.CUST_SSN==ssn).\
        filter((spark_df.TIMEID==first_date)&(spark_df.TIMEID==second_date))
        #sort(spark_df.year.desc(),spark_df.month.desc(),spark_df.day.desc())
    mod4.show(10)

          

In [ ]:
module_four()

In [231]:
def module_four():
    # todo get user input
    print('Module Four:\n\t Transactions made by a customer between two dates.')
    ccn = 4210653369905302
    start_day = 3
    start_month = 11
    start_year = 2018
    end_day = 22
    end_month = 12
    end_year = 2018

    mod4 = spark_df.select('CUST_CC_NO', 'year', 'month', 'days', 'TRANSACTION_TYPE', 'TRANSACTION_VALUE'). \
        filter(spark_df.CUST_CC_NO == ccn). \
        filter(spark_df.year >= start_year). \
        filter(spark_df.month >= start_month). \
        filter(spark_df.days >= start_day). \
        filter(spark_df.year <= end_year). \
        filter(spark_df.month<= end_month). \
        filter(spark_df.days <= end_day). \
        sort(spark_df.year.desc(), spark_df.month.desc(), spark_df.days.desc())
    mod4.show(100)

In [232]:
module_four()

Module Four:
	 Transactions made by a customer between two dates.
+----------------+----+-----+----+----------------+-----------------+
|      CUST_CC_NO|year|month|days|TRANSACTION_TYPE|TRANSACTION_VALUE|
+----------------+----+-----+----+----------------+-----------------+
|4210653369905302|2018|   12|  21|         Grocery|            47.08|
|4210653369905302|2018|   12|  20|   Entertainment|            25.49|
|4210653369905302|2018|   12|  15|             Gas|            100.3|
|4210653369905302|2018|   11|  12|       Education|            63.23|
|4210653369905302|2018|   11|  11|         Grocery|            47.01|
|4210653369905302|2018|   11|   9|       Education|             61.3|
|4210653369905302|2018|   11|   9|             Gas|            41.48|
|4210653369905302|2018|   11|   3|            Test|            19.79|
+----------------+----+-----+----+----------------+-----------------+

